In [2]:
import ase
from ase.build import fcc111, fcc100, fcc110
from moleidoscope.visualize import show
from moleidoscope.linker import Linker
from xyz2cif import write_cif

In [3]:
def ase_to_linker(ase_atoms):
    """ Convert ASE atoms object to Moleidoscope linker object """
    mol = Linker()
    mol.atom_names = list(ase_atoms.get_chemical_symbols())
    mol.atom_coors = list(ase_atoms.get_positions())
    return mol

# Parameters

In [4]:
slab_packing = (10, 15, 5)    # Packing of metal slab
vacuum = 0.0                  # Vacuum on top of slab (ASE)
z_box = 50                    # Length of simulation box in z-direction (A)
distance_from_surface = 5     # Distance btw molecule and surface (A)

## Cu (110) Slab

In [5]:
cu_slab = fcc110('Cu', a=3.908, size=slab_packing, vacuum=vacuum)
cu110 = ase_to_linker(cu_slab)
print(cu_slab.cell)

[[ 39.08         0.           0.        ]
 [  0.          41.45059951   0.        ]
 [  0.           0.           5.5267466 ]]


In [6]:
# show(cu110)

### Aligning Decacylene (*xy-plane*)
Original xyz file already has one head (atoms 7 & 8) aligned to *y-axis*.

Here I am using the vector between the two opposing atoms in the middle benzene ring that is perpendicular to the head on *y-axis*.

By aligning this vector to *x-axis* the molecule is now aligned to *xy-plane*.

In [7]:
decacylene = Linker(read='DC-single.xyz')            # Read molecule
decacylene.vector = decacylene.get_vector(0, 1)      # Get vector for middle benzene ring atoms
dc_align = decacylene.align([1, 0, 0])               # Align molecule on x-axis
dc_align.center([0, 0, 0])                           # Center molecule to origin

In [8]:
# show(dc_align, decacylene, distance=[20, 0, 0])

In [8]:
# dc_align.name='DC-align'           # Rename molecule
# dc_align.save(file_format='xyz')   # Save as xyz file

### Placing aligned decacylene on Cu (110) surface
I will place the aligned decacylene 5 Å above the Cu (110) surface.

In [9]:
slab_center = cu110.get_center()
dc_align.center(slab_center)
dc_align.translate([0, 0, distance_from_surface + cu_slab.cell[2][2] / 2])
cu110_dc = cu110.join(dc_align)

In [11]:
show(cu110_dc)

The installed widget Javascript is the wrong version.


In [17]:
# cu110_dc.name='DC_Cu110'           # Rename molecule
# cu110_dc.save(file_format='xyz')   # Save as xyz file

## Create cif file

In [18]:
# 'BTW_FF', 'Dreiding', 'UFF', 'UFF4MOF', and 'Dubbeldam'
force_field = 'Dubbeldam'
sim_box = [cu_slab.cell[0][0], cu_slab.cell[1][1], z_box, 90, 90, 90]
# cif_file = 'DC_Cu110_%s.cif' % '_'.join([str(int(i)) for i in sim_box[:3]])
cif_file = 'DC_Cu110_%s.cif' % force_field
write_cif(cif_file, cu110_dc.atom_names, cu110_dc.atom_coors, header=cif_file, cell=sim_box, fractional=True)

## Create Lammps simulation files

In [19]:
import os
from thermof import Simulation
from thermof import Parameters

In [20]:
simpar = Parameters()
sim = Simulation(mof=cif_file, parameters=simpar)
mof_name = os.path.splitext(os.path.basename(cif_file))[0]
sim.simdir = os.path.join(os.path.dirname(cif_file), mof_name)

sim.parameters.lammps['force_field'] = force_field
sim.parameters.lammps['mol_ff'] = force_field
sim.parameters.thermof['fix'] = ['NVT', 'NVE']
sim.parameters.thermof['temperature'] = 298
sim.parameters.thermof['thermo_style'] = ['step', 'temp', 'press', 'pe', 'etotal', 'epair', 'emol']
sim.parameters.job['scheduler'] = 'slurm'
sim.parameters.job['cluster'] = 'smp'
sim.parameters.job['nodes'] = 1
sim.parameters.job['ppn'] = 4

In [ ]:
sim.initialize()

I. Writing Lammps input and data files...
No bonds reported in cif file - computing bonding..
Detecting Inorganic clusters
